In [2]:
!pip3 install numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 15.8 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.3 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 1.9 MB/s eta 0:00:01


In [31]:
import numpy as np
import matplotlib.pyplot as plt

def normalize(vector):
    return vector / np.linalg.norm(vector)

def reflected(vector, axis):
#     print(2 * np.dot(vector, axis))
    return vector - 2 * np.dot(vector, axis) * axis

def sphere_intersect(center, radius, ray_origin, ray_direction):
    b = 2 * np.dot(ray_direction, ray_origin - center)
    c = np.linalg.norm(ray_origin - center) ** 2 - radius ** 2
    delta = b ** 2 - 4 * c
    if delta > 0:
        t1 = (-b + np.sqrt(delta)) / 2
        t2 = (-b - np.sqrt(delta)) / 2
        if t1 > 0 and t2 > 0:
            return min(t1, t2)
    return None

def nearest_intersected_object(objects, ray_origin, ray_direction):
    distances = [sphere_intersect(obj['center'], obj['radius'], ray_origin, ray_direction) for obj in objects]
    nearest_object = None
    min_distance = np.inf
    for index, distance in enumerate(distances):
        if distance and distance < min_distance:
            min_distance = distance
            nearest_object = objects[index]
    return nearest_object, min_distance

width = 10
height = 10

max_depth = 3

camera = np.array([0, 0, 1])
ratio = float(width) / height
screen = (-1, 1 / ratio, 1, -1 / ratio) # left, top, right, bottom

light = { 'position': np.array([5, 5, 5]), 'ambient': np.array([1, 1, 1]), 'diffuse': np.array([1, 1, 1]), 'specular': np.array([1, 1, 1]) }

objects = [
    { 'center': np.array([-0.2, 0, -1]), 'radius': 0.7, 'ambient': np.array([0.1, 0, 0]), 'diffuse': np.array([0.7, 0, 0]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 },
    { 'center': np.array([0.1, -0.3, 0]), 'radius': 0.1, 'ambient': np.array([0.1, 0, 0.1]), 'diffuse': np.array([0.7, 0, 0.7]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 },
    { 'center': np.array([-0.3, 0, 0]), 'radius': 0.15, 'ambient': np.array([0, 0.1, 0]), 'diffuse': np.array([0, 0.6, 0]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 },
    { 'center': np.array([0, -9000, 0]), 'radius': 9000 - 0.7, 'ambient': np.array([0.1, 0.1, 0.1]), 'diffuse': np.array([0.6, 0.6, 0.6]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 }
]

image = np.zeros((height, width, 3))
for i, y in enumerate(np.linspace(screen[1], screen[3], height)):
    for j, x in enumerate(np.linspace(screen[0], screen[2], width)):
        # screen is on origin
        pixel = np.array([x, y, 0])
        origin = camera
        direction = normalize(pixel - origin)

        color = np.zeros((3))
        reflection = 1

        for k in range(max_depth):
            # check for intersections
            nearest_object, min_distance = nearest_intersected_object(objects, origin, direction)
            if nearest_object is None:
#                 print('skipping', k, i, j)
                break

            intersection = origin + min_distance * direction
            normal_to_surface = normalize(intersection - nearest_object['center'])
            shifted_point = intersection + 1e-5 * normal_to_surface
            intersection_to_light = normalize(light['position'] - shifted_point)
            
            if k == 0 and i == 3 and j == 4:
                print("is:", intersection)
                print("nts:", normal_to_surface)
                print("sp:", shifted_point)
                print("itl:", intersection_to_light)

            _, min_distance = nearest_intersected_object(objects, shifted_point, intersection_to_light)
            intersection_to_light_distance = np.linalg.norm(light['position'] - intersection)
            is_shadowed = min_distance < intersection_to_light_distance
            print('abcd:', min_distance, intersection_to_light_distance)
            if is_shadowed:
                break

            illumination = np.zeros((3))

            # ambiant
            illumination += nearest_object['ambient'] * light['ambient']
            print('i1:', illumination)
            # diffuse
            illumination += nearest_object['diffuse'] * light['diffuse'] * np.dot(intersection_to_light, normal_to_surface)
            print('i2:', illumination)

            # specular
            intersection_to_camera = normalize(camera - intersection)
            H = normalize(intersection_to_light + intersection_to_camera)
            print('specD:', np.dot(normal_to_surface, H) ** (nearest_object['shininess'] / 4))
            print('specI:', nearest_object['specular'] * light['specular'])
            illumination += nearest_object['specular'] * light['specular'] * np.dot(normal_to_surface, H) ** (nearest_object['shininess'] / 4)
            print('itc:', intersection_to_camera)
            print('H:', H)
            print('i3:', illumination)

            # reflection
            color += reflection * illumination
            reflection *= nearest_object['reflection']
            
            print('color:', color)
            print('reflection:', reflection)

            origin = shifted_point
            direction = reflected(direction, normal_to_surface)

        image[i, j] = np.clip(color, 0, 1)
    print("%d/%d" % (i + 1, height))

plt.imsave('image.png', image)

1/10
2/10
3/10
is: [-0.16855145  0.50565436 -0.51696309]
nts: [0.04492649 0.72236338 0.69005272]
sp: [-0.16855101  0.50566159 -0.51695619]
itl: [0.58767769 0.51101796 0.62729226]
abcd: inf 8.794881955604264
i1: [0.1 0.  0. ]
i2: [0.67988538 0.         0.        ]
specD: 0.00035123304376618534
specI: [1 1 1]
itc: [ 0.10482848 -0.31448545  0.94345635]
H: [0.40079233 0.11374443 0.9090807 ]
i3: [6.80236617e-01 3.51233044e-04 3.51233044e-04]
color: [6.80236617e-01 3.51233044e-04 3.51233044e-04]
reflection: 0.5
4/10
abcd: inf 8.714991086919591
i1: [0.  0.1 0. ]
i2: [0.         0.58569856 0.        ]
specD: 0.003924216154898087
specI: [1 1 1]
itc: [ 0.31448545 -0.10482848  0.94345635]
H: [0.50590665 0.25104319 0.82524892]
i3: [0.00392422 0.58962278 0.00392422]
color: [0.00392422 0.58962278 0.00392422]
reflection: 0.5
abcd: inf 8.849832884565718
i1: [0.1 0.  0. ]
i2: [0.62150429 0.         0.        ]
specD: 0.2599234029293845
specI: [1 1 1]
itc: [ 0.10976426 -0.10976426  0.98787834]
H: [0.386

In [5]:
objects = [
    { 'center': np.array([-0.2, 0, -1]), 'radius': 0.7, 'ambient': np.array([0.1, 0, 0]), 'diffuse': np.array([0.7, 0, 0]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 },
    { 'center': np.array([0.1, -0.3, 0]), 'radius': 0.1, 'ambient': np.array([0.1, 0, 0.1]), 'diffuse': np.array([0.7, 0, 0.7]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 },
    { 'center': np.array([-0.3, 0, 0]), 'radius': 0.15, 'ambient': np.array([0, 0.1, 0]), 'diffuse': np.array([0, 0.6, 0]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 },
    { 'center': np.array([0, -9000, 0]), 'radius': 9000 - 0.7, 'ambient': np.array([0.1, 0.1, 0.1]), 'diffuse': np.array([0.6, 0.6, 0.6]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection': 0.5 }
]

nearest_intersected_object(objects, [0, 0, 1], [-0.10482848,  0.31448545, -0.94345635])

({'center': array([-0.2,  0. , -1. ]),
  'radius': 0.7,
  'ambient': array([0.1, 0. , 0. ]),
  'diffuse': array([0.7, 0. , 0. ]),
  'specular': array([1, 1, 1]),
  'shininess': 100,
  'reflection': 0.5},
 1.60787843946078)

In [6]:
np.dot(v, a)

6

In [15]:
np.linspace(screen[1], screen[3], height)

array([ 1.        ,  0.77777778,  0.55555556,  0.33333333,  0.11111111,
       -0.11111111, -0.33333333, -0.55555556, -0.77777778, -1.        ])

In [16]:
screen[1]

1.0